In [20]:
import json
import nltk
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load JSON file with rules
def load_rules(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

rules_data = load_rules("ruledictionary_H1_fields.txt")

# Extract validation rules into a DataFrame
rules = []
for field, details in rules_data["Corporate Loan Data Fields"].items():
    rules.append({
        "Field": field,
        "Condition": details["validation"],
        "Valid": 1,
        "Valid_Example": details["valid_example"],
        "Invalid_Example": details["invalid_example"]
    })

df = pd.DataFrame(rules)

# Feature extraction
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def extract_features(text, valid_example, invalid_example):
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    return {
        "num_tokens": len(tokens),
        "num_nouns": len([word for word, pos in pos_tags if pos.startswith("NN")]),
        "num_verbs": len([word for word, pos in pos_tags if pos.startswith("VB")]),
        "valid_example_length": len(valid_example) if isinstance(valid_example, str) else 0,
        "invalid_example_count": len(invalid_example) if isinstance(invalid_example, list) else 0
    }

features_df = pd.DataFrame(df.apply(lambda row: extract_features(str(row['Condition']), row['Valid_Example'], row['Invalid_Example']), axis=1).tolist())
X_train, X_test, y_train, y_test = train_test_split(features_df, df['Valid'], test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Print model accuracy
accuracy = clf.score(X_test, y_test)
print(f"Validation Rule Classifier Accuracy: {accuracy:.2%}")

# Function to validate individual field values
def validate_input(field_name, value, internal_id=None):
    errors = []
    field_info = rules_data["Corporate Loan Data Fields"].get(field_name, {})
    
    if not field_info:
        return "Error: No validation rules found."
    
    validation_rule = field_info.get("validation", "")
    valid_example = field_info.get("valid_example", "No example available.")
    invalid_example = field_info.get("invalid_example", ["No invalid examples."])
    
    if not value or pd.isna(value):
        errors.append(f"'{field_name}' cannot be empty.")
    
    value = str(value).strip()
    
    if "must be a whole number" in validation_rule.lower() and not re.fullmatch(r"^-?\d+$", value):
        errors.append(f"'{value}' must be a valid whole number.")
    if "YYYY-MM-DD" in validation_rule and not re.fullmatch(r"^\d{4}-\d{2}-\d{2}$", value):
        errors.append(f"'{value}' must be in YYYY-MM-DD format.")
    if "must be uppercase" in validation_rule.lower() and not value.isupper():
        errors.append(f"'{value}' must be uppercase.")
    if "max 20 characters" in validation_rule.lower() and len(value) > 20:
        errors.append(f"'{value}' exceeds the max length of 20 characters.")
    if "must be uppercase" in validation_rule.lower() and not value.isupper():
        errors.append(f"'{value}' must be uppercase.")
    
    if "must be exactly 2 characters" in validation_rule.lower() and not re.fullmatch(r"^[A-Z]{2}$", value):
        errors.append(f"'{value}' must be exactly 2 characters (ISO 3166-1 alpha-2 country code).")
    
    if "YYYY-MM-DD" in validation_rule and not re.fullmatch(r"^\d{4}-\d{2}-\d{2}$", value):
        errors.append(f"'{value}' must be in YYYY-MM-DD format.")
    
    if "must be a future date" in validation_rule and value <= "2024-03-26":
        errors.append(f"'{value}' must be a future date.")
    
    if "must be a past or present date" in validation_rule and value > "2024-03-26":
        errors.append(f"'{value}' must be a past or present date.")
    
    if "decimal format" in validation_rule.lower() and not re.fullmatch(r"^\d+(\.\d+)?$", value):
        errors.append(f"'{value}' must be a valid decimal number.")
    
    if "percentage format" in validation_rule.lower() and not re.fullmatch(r"^\d{1,3}(\.\d{1,2})?$", value):
        errors.append(f"'{value}' must be a valid percentage (0-100, max 2 decimals).")
   
    if field_name == "Country" and not re.fullmatch(r"^[A-Z]{2}$", value):
        errors.append(f"'{value}' must be a valid 2-letter country code (ISO 3166-1). Example: 'US'.")
    
    if "must be alphanumeric" in validation_rule.lower() and not re.fullmatch(r"^[a-zA-Z0-9]+$", value):
        errors.append(f"'{value}' must be alphanumeric, with no special characters.")
    
    if "max 20 characters" in validation_rule.lower() and len(value) > 20:
        errors.append(f"'{value}' exceeds the maximum length of 20 characters.")   
    
    if "must match format of Internal_ID" in validation_rule.lower() and internal_id is not None:
        if not re.fullmatch(r"^[a-zA-Z0-9]+$", value) or len(value) != len(internal_id):
            errors.append(f"'{value}' must match the format and length of Internal_ID '{internal_id}'.")
    if "cannot contain spaces or special characters" in validation_rule.lower() and not re.fullmatch(r"^[a-zA-Z0-9]+$", value):
        errors.append(f"'{value}' cannot contain spaces or special characters.")
    if "max 50 characters" in validation_rule.lower() and len(value) > 50:
        errors.append(f"'{value}' exceeds the maximum length of 50 characters.")
    if "max 10 characters" in validation_rule.lower() and len(value) > 10:
        errors.append(f"'{value}' exceeds the maximum length of 10 characters.")    
    if "max 100 characters" in validation_rule.lower() and len(value) > 100:
        errors.append(f"'{value}' exceeds the maximum length of 100 characters.")
    if "must be a whole number" in validation_rule.lower():
        if not re.fullmatch(r"^\d+$", value):
            errors.append(f"'{value}' must be a whole number with no decimals, commas, or symbols.")
        elif int(value) < 0:
            errors.append(f"'{value}' cannot be negative.")
    if "decimal format" in validation_rule.lower():
        if not re.fullmatch(r"^0(\.\d{1,7})?$|^1(\.0{1,7})?$", value):
            errors.append(f"'{value}' must be a decimal number between 0 and 1 with up to 7 decimal places.")
        elif not (0 <= float(value) <= 1):
            errors.append(f"'{value}' must be between 0 and 1.")
    if "decimal format" in validation_rule.lower():
        if not re.fullmatch(r"^\d{1,3}(\.\d{1,4})?$", value):
            errors.append(f"'{value}' must be a decimal number with up to 4 decimal places, between 0 and 100.")
        elif not (0 <= float(value) <= 100):
            errors.append(f"'{value}' must be between 0 and 100.")
    if field_name == "Currency" and not re.fullmatch(r"^[A-Z]{3}$", value):
        errors.append(f"'{value}' must be a valid ISO 4217 three-letter currency code in uppercase.")
    if field_name == "Currency" and not re.fullmatch(r"^[A-Z]{3}$", value):
        errors.append(f"'{value}' must be a valid ISO 4217 three-letter currency code in uppercase.")
    if "enumerated values" in validation_rule.lower():
            valid_options = re.findall(r"\((.*?)\)", validation_rule)
            if valid_options:
                valid_options = [opt.strip() for opt in valid_options[0].split(",")]
                if value not in valid_options:
                    errors.append(f"'{value}' is not a valid option. Allowed values: {', '.join(valid_options)}.")
    if errors:
        return "Non-Compliant", errors
    
    # Machine Learning validation
    input_features = extract_features(validation_rule, valid_example, invalid_example)
    input_df = pd.DataFrame([input_features])
    prediction = clf.predict(input_df)[0]
    
    return ("Compliant" if prediction == 1 else "Non-Compliant (ML Prediction)", [])

# Function to validate an entire XLS file
def validate_xls(file_path):
    df = pd.read_excel(file_path)
    anomalies = {col: [] for col in df.columns}
    
    for _, row in df.iterrows():
        for field_name, value in row.items():
            compliance_status, issues = validate_input(field_name, value)
            anomalies[field_name].append((value, compliance_status, "; ".join(issues)))
    
    anomalies_df = pd.DataFrame({col: [list(x) for x in zip(*values)] for col, values in anomalies.items()},
                                index=["Value", "Compliance_Status", "Issues"]).T
    
    anomalies_df.to_excel("anomalies_report.xlsx", index=True)
    anomalies_df.to_json("anomalies_report.json", orient="records", indent=4)
    return "Validation completed. Report saved to anomalies_report.xlsx and json"

print(validate_xls("sample_datavalidation_withmodel.xlsx"))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Hi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Validation Rule Classifier Accuracy: 100.00%
Validation completed. Report saved to anomalies_report.xlsx and json
